### 2021 Spring "EE479: Scientific Computing & Data"
#### Hyeonwoo Yeo, KAIST Electrical Engineering

#### 1. Solving time-independent one-dimensional (1-D) Schrödinger equation using matrix diagonalization

###### Last updated : 2024. 03. 16

###### To solve:
 $ -\dfrac{\hbar^2}{2m} \, \dfrac{\mathrm{d}^2 \psi}{\mathrm{d} x^2} + V(x)\psi = H\psi = E\psi $ ( $ Ax = \lambda x $ )
 
###### Numerical implementation:


1. Define Laplacian using FDM and external Potential for 1-D Hamiltonian.

2. Construct 1-D Hamiltonian using the information from step 1.

3. Diagonalize the Hamiltonian using numpy.eigh and get eigenvalues ($E_n$) and eigenvectors ($\psi$).

4. Visualize with interaction.


In [16]:
import numpy as np
import numpy.linalg as lin
from matplotlib import pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed

# Define FDM points & coefficients (From the definition in wikipedia)
def fdmcoefficient(l):
    a=np.zeros((2*l+1,2*l+1))
    b=np.zeros(2*l+1)
    c=np.zeros(2*l+1)

    for i in range(0, 2*l+1):
        for j in range(0, 2*l+1):
            a[i,j]= (j-l)**i
    c[2]=2*1   #2!
    a = lin.inv(a)
    b= np.matmul(a, c)

    C=np.zeros((l+1))

    for i in range(0, l+1):
        C[i]=b[i+l]
    return C

# Define the Laplacian for Hamiltonian
def Laplacian(n, l, dx):
    C=fdmcoefficient(l)
    oprt=np.zeros((n,n))
    for i in range(n):
        for j in range(-l, l + 1, 1):
            k = i + j
            if (k >= n):
                k -= n
                oprt[i][k] = C[abs(j)] / (dx**2)
            elif (k<0):
                k += n
                oprt[i][k] = C[abs(j)] / (dx**2)
            else:
                oprt[i][k] = C[abs(j)] / (dx**2)
    return oprt

# Set shapes of potential
def Potential(L,n, pot_shape=0, pot_height_eV=25):
    pot_height_har = pot_height_eV/27.211
    grd=np.zeros(n)
    grd[:]=0
        
    grd[0] = 1000000000
    grd[n-1]=1000000000
     
    if pot_shape == 0:
        for i in range(1, n-1):
            grd[i] = grd[i]
            
    if pot_shape == 1:                               # Harmonic
        for i in range(1, n-1):
            grd[i]=(i-(n-1)//2)**2/((n-1)//2)**2
            grd[i]=grd[i]*pot_height_har
                  
    if pot_shape == 2:                               # Square well 
        for i in range(1,(4*n)//10):
            grd[i] = pot_height_har
        for i in range((4*n)//10,(6*n)//10+1):
            grd[i] = 0
        for i in range((6*n)//10+1,n-1):
            grd[i] = pot_height_har
        
    if pot_shape == 3:                               #Triangular
        grd[:]=10**6
        for i in range((5*n)//10,n):
            grd[i] = pot_height_har*abs(i-500)/500

    if pot_shape == 4:                              #Asymmetric infinite Well
        for i in range((5*n//10), n-2):
            grd[i] = pot_height_har                 # eV unit

    return grd

In [28]:
# Define Hamiltonian using Potential and Laplacian
def Hamiltonian(L, n, l, dx, pot_height_eV=25, pot_shape=0):
    Hamiltonian = np.zeros((n,n))
    V = Potential(L, n, pot_shape, pot_height_eV)
    L = Laplacian(n, l, dx)                   # h- = 1, m_e = 1
    V_oprt=np.zeros((n,n))
    for i in range(0, n):
        V_oprt[i,i]=V[i]
    Hamiltonian = -L / 2. + V_oprt    # H = - (h-^2/2m) L + V
    return Hamiltonian

# Construct Hamiltonian using Potential and Laplacian
def TISE(n, m, e, s, t, pot_height_eV=25, pot_shape=0, nstate=5):
    
    #Input & unit conversion section
    Lx = 100                         # Box Length (Angstrom = 10^-10 m)
    ngx = 1001                       # Number of grid points.  (spacing = L/(n-1))      
    l = 3
    Lx = Lx * 1.88973
    dx = Lx / ngx
    pot_height_eV = 25               # Height of potential in eV

    #Making Hamiltonian Matrix
    H = np.zeros((ngx, ngx))
    dn = np.ones(ngx) / dx**2
    dno = np.ones(ngx-1) * (-1/2) / dx**2

    H = Hamiltonian(Lx, ngx, l, dx, pot_height_eV=pot_height_eV, pot_shape=pot_shape)

    (w, v) = np.linalg.eigh(H)

    a = np.linspace(-Lx/2, Lx/2, ngx)
    a = a / 1.89

    # Print Eigenvalues
    for i in range(0, nstate):
        print ('%dth state eigenvalue is %f eV' %(i+1, w[i]*27.211))

    # Plot eigenstates
    for i in range(0, nstate):
        height = 0.08 * max(w[:nstate]) * 27.211 * 2    # Height constante of eigenvectors for visualization
        len_height = np.max(v[:,i]) - np.min(v[:,i])
        eigenstate = v[:, i] / len_height * height + np.ones(ngx) * w[i] * 27.211
        plt.plot(a, eigenstate)

    # Dotted line for eigenvalues
    for i in range(0, nstate):
        plt.axhline(y=w[i] * 27.211, color='gray', linestyle='--', linewidth=0.7)  
        
    # Adjust y-axis range to show eigenvalues clearly
    plt.ylim(0, max(w[:nstate]) * 27.211 * 2)

    V = Potential(Lx, ngx, pot_shape, pot_height_eV)
    plt.plot(a, V*27.211, color='k')
    plt.ylabel('Energy [eV]')
    plt.xlabel('Box [Angstrom]')
    plt.show()

    # Interactive part for selecting state
    @interact(selected_state=widgets.Dropdown(options=list(range(1, nstate+1)), value=1, description='State for probability density: ', style={'description_width': 'initial'}))
    def plot_probability_density(selected_state):
        plt.figure()
        prob_density = (v[:, selected_state-1]/len_height) ** 2
        plt.plot(a, prob_density, color='black')
        plt.fill_between(a, prob_density, color='lightgrey')
        plt.ylim(0, 1)
        plt.ylabel('Probability density ($|\Psi|^2$) \n(A.U)', labelpad=20)
        plt.xlabel('Box [Angstrom]')
        plt.text(0.5, 0.9, '$\int_{-\infty}^{\infty}|\Psi|^2\,dx = 1$', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes, fontsize=10)  
        plt.tick_params(axis='y', which='both', left=False, labelleft=False) 
        plt.show()

# Interactive part
if __name__=="__main__":
    @interact(nstate=widgets.IntSlider(min=1, max=10, step=1, value=5, description='# of States: '),
              pot_shape=widgets.Dropdown(options=[('Particle in a box (Infinite well)', 0), 
                                                  ('Harmonic Oscillator', 1), 
                                                  ('Square Well', 2), 
                                                  ('Triangular Well', 3), 
                                                  ('Asymmetric infinite Well', 4)], 
                                          value=0, 
                                          description='Potential Shape: ', style={'description_width': 'initial'}))
    def interactive(nstate, pot_shape):
        TISE(4, 0, 0, 0, 0, pot_shape=pot_shape, nstate=nstate)


interactive(children=(IntSlider(value=5, description='# of States: ', max=10, min=1), Dropdown(description='Po…

### Example: analytical solution for infinite potential problem :

$ E_n = $ $ \hbar^2(n\pi)^2 \over 2mL^2 $

For L = 100Å

$ E_1 = 0.003760 eV$  

$ E_2 = 0.015004 eV$  

$ E_3 = 0.033843 eV$  

$ E_4 = 0.060164 eV$  

$ E_5 = 0.094007 eV$